In [4]:
# Importing libraries
import pandas as pd
import wget
import numpy as np
import pygsheets
from IPython.display import display, HTML

In [5]:
# If you want to download the files and access them locally, you can do it this way.

# Adding URL's of the CSV Dataset
# urls = [
#     "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv",
#     "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv",
#     "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
# ]

# Downloading and Saving CSV files locally
# [wget.download(i) for i in urls]

In [6]:
# Storing the Raw CSV Dataset(s) into a DataFrame
confirmed_df = pd.read_csv(r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths_df = pd.read_csv(r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered_df = pd.read_csv(r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34194,34366,34451,34455,34740,34994,35070,35229,35301,35475
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3278,3371,3454,3571,3667,3752,3851,3906,4008,4090
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,18242,18712,19195,19689,20216,20770,21355,21948,22549,23084
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,855,855,855,858,861,862,877,880,880,880
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,458,462,506,525,541,576,607,638,687,705


### Now that we have had a look on the data, we can clearly define some problems and how we are going to fix them:

#### Problems:
1. Confirmed, Deaths and Recovered Cases are stored in three different datasets, which can make it difficult to visualize data
2. Dates exist as columns and give a very odd shape to the DataFrame.
3. There is also some missing data and some wrong Datatypes.

#### Fixes:
1. Merge the Confirmed, Deaths and Recovered DataFrames into a Single DataFrame.
2. Extracting the common date list from the dataset and Transpose the dates into values using the melt() function. 
3. Perform Data Cleaning for missing values and fixing wrong datatypes.
4. Add a new column namely _Active Cases_ which is calculated as **Active Cases = Confirmed - Deaths - Recovered**.
5. Aggregate Data Country/Region Wise and Group them by Date and Country/Region.
6. Add New columns **New Cases, New Deaths and New Recovered** by subtracting the Cumulative Data from the previous day.

In [7]:
# Storing the common date list in a variable.
dates = confirmed_df.columns[4:]

# Melting/Unpivoting the DataFrames from the current wide format to a long format.

confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    value_vars= dates,
    var_name='Date',
    value_name='Confirmed'
)

deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    value_vars= dates,
    var_name='Date',
    value_name='Deaths'
)

recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    value_vars= dates,
    var_name='Date',
    value_name='Recovered'
)

confirmed_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
47875,NaN,Sao Tome and Principe,0.186400,6.613100,7/19/20,746
47876,NaN,Yemen,15.552727,48.516388,7/19/20,1606
47877,NaN,Comoros,-11.645500,43.333300,7/19/20,334
47878,NaN,Tajikistan,38.861000,71.276100,7/19/20,6878


In [8]:
# Merging the DataFrames into a Single DataFrame

# Merging confirmed_df and deaths_df
final_table = confirmed_df_long.merge(
    right= deaths_df_long,
    how='left',
    on=['Province/State', 'Country/Region', 'Lat', 'Long', 'Date']
)

# Merging final_table and recovered_df
final_table = final_table.merge(
    right= recovered_df_long,
    how='left',
    on=['Province/State', 'Country/Region', 'Lat', 'Long', 'Date']
)

final_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
47875,NaN,Sao Tome and Principe,0.186400,6.613100,7/19/20,746,14,451.0
47876,NaN,Yemen,15.552727,48.516388,7/19/20,1606,445,712.0
47877,NaN,Comoros,-11.645500,43.333300,7/19/20,334,7,313.0
47878,NaN,Tajikistan,38.861000,71.276100,7/19/20,6878,57,5578.0


In [9]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47880 entries, 0 to 47879
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  14580 non-null  object 
 1   Country/Region  47880 non-null  object 
 2   Lat             47880 non-null  float64
 3   Long            47880 non-null  float64
 4   Date            47880 non-null  object 
 5   Confirmed       47880 non-null  int64  
 6   Deaths          47880 non-null  int64  
 7   Recovered       43380 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 3.3+ MB


### Data Cleaning
Looking at the final data, there are still some things left to do make the data fit for use
1. Replace Missing Values
2. Convert Date from String object to DateTime
3. Removing Data gathered from Cruise Ships.

We can see a lot of NaN values in the **Province/State** Column, justifiably so as a lot of counties don't report the data State/Province wise. So, we will leave that as it is for now. However, there are quite a few NaN values in the **Recovered** Column. Let's Replace those values with 0.

In [10]:
final_table['Recovered'] =  final_table['Recovered'].fillna(0)

# Converting Date from String to DateTime
final_table['Date'] = pd.to_datetime(final_table['Date'])

final_table.sort_values(by=['Country/Region', 'Date'], inplace=True)

final_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
266,NaN,Afghanistan,33.939110,67.709953,2020-01-23,0,0,0.0
532,NaN,Afghanistan,33.939110,67.709953,2020-01-24,0,0,0.0
798,NaN,Afghanistan,33.939110,67.709953,2020-01-25,0,0,0.0
1064,NaN,Afghanistan,33.939110,67.709953,2020-01-26,0,0,0.0
...,...,...,...,...,...,...,...,...
46780,NaN,Zimbabwe,-19.015438,29.154857,2020-07-15,1089,20,395.0
47046,NaN,Zimbabwe,-19.015438,29.154857,2020-07-16,1362,23,425.0
47312,NaN,Zimbabwe,-19.015438,29.154857,2020-07-17,1420,24,438.0
47578,NaN,Zimbabwe,-19.015438,29.154857,2020-07-18,1478,25,439.0


In [11]:
# Filtering out the data from Cruise Ships
ship_rows = final_table['Province/State'].str.contains('Grand Princess') | final_table['Province/State'].str.contains('Diamond Princess') | final_table['Country/Region'].str.contains('Diamond Princess') | final_table['Country/Region'].str.contains('MS Zaandam')
full_ship = final_table[ship_rows]

# Altering the table to remove this data
final_table = final_table[~(ship_rows)]

### Data Aggregation

Now that we have our cleaned data, we will transform it such that it can give us some valuable insights and summaries.
The first thing to do is to add another column, namely **Active** Cases.

In [12]:
final_table['Active'] = final_table['Confirmed'] - final_table['Recovered'] - final_table['Deaths']

final_table

C:\Users\jaspr\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0,0.0
266,NaN,Afghanistan,33.939110,67.709953,2020-01-23,0,0,0.0,0.0
532,NaN,Afghanistan,33.939110,67.709953,2020-01-24,0,0,0.0,0.0
798,NaN,Afghanistan,33.939110,67.709953,2020-01-25,0,0,0.0,0.0
1064,NaN,Afghanistan,33.939110,67.709953,2020-01-26,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
46780,NaN,Zimbabwe,-19.015438,29.154857,2020-07-15,1089,20,395.0,674.0
47046,NaN,Zimbabwe,-19.015438,29.154857,2020-07-16,1362,23,425.0,914.0
47312,NaN,Zimbabwe,-19.015438,29.154857,2020-07-17,1420,24,438.0,958.0
47578,NaN,Zimbabwe,-19.015438,29.154857,2020-07-18,1478,25,439.0,1014.0


Now, let's group our data by 'Date' and 'Country/Region'.

In [13]:
final_grouped = final_table.groupby(['Date', 'Country/Region'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()

# sum() is used here to get the total for 'Confirmed', 'Deaths', 'Recovered' and 'Active' Cases for that given Date.

final_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
33475,2020-07-19,West Bank and Gaza,8549,62,1921.0,6566.0
33476,2020-07-19,Western Sahara,10,1,8.0,1.0
33477,2020-07-19,Yemen,1606,445,712.0,449.0
33478,2020-07-19,Zambia,2980,120,1462.0,1398.0


In [14]:
# Creating a temp DataFrame for calculating New Values
temp = final_table.groupby(['Country/Region', 'Date'])[['Confirmed', 'Deaths', 'Recovered']]
temp = temp.sum().diff().reset_index()

# diff() helps us calculate the diffrence of a cell with it's preceding cell

mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

# The mask variable will help us root out the wrong values in a case where the difference is calculated between
# different countries and not between the same country for different dates

temp.loc[mask, ['Confirmed', 'Deaths', 'Recovered']] = np.nan

# Renaming Columns
temp.columns = ['Country/Region', 'Date', 'New Cases', 'New Deaths', 'New Recovered']

# Merging the DataFrames
final_grouped = pd.merge(left=final_grouped, right=temp, on=['Country/Region', 'Date'])

In [15]:
final_grouped.fillna(0, inplace=True)

# Fixing Data types
final_grouped[['Recovered', 'Active', 'New Cases', 'New Deaths', 'New Recovered']] = final_grouped[['Recovered', 'Active', 'New Cases', 'New Deaths', 'New Recovered']].astype('int')

# Fixing negative Values in the data
final_grouped['New Cases'] = final_grouped['New Cases'].apply(lambda x: 0 if x < 0 else x)

final_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0
1,2020-01-22,Albania,0,0,0,0,0,0,0
2,2020-01-22,Algeria,0,0,0,0,0,0,0
3,2020-01-22,Andorra,0,0,0,0,0,0,0
4,2020-01-22,Angola,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
33475,2020-07-19,West Bank and Gaza,8549,62,1921,6566,345,3,325
33476,2020-07-19,Western Sahara,10,1,8,1,0,0,0
33477,2020-07-19,Yemen,1606,445,712,449,25,2,11
33478,2020-07-19,Zambia,2980,120,1462,1398,0,0,0


In [16]:
client = pygsheets.authorize(service_account_file=r'D:\Python\Notebooks\Covid 19 Dashboard\Covid 19 Dashboard-7fcdf6851df8.json')
sheet = client.open('Covid 19 Data with Auto Refresh')

wks = sheet[0]

# wks.rows = final_grouped[0]
wks.set_dataframe(final_grouped, start=(1,1), fit=True)

### Here is the final version of the interactive Covid 19 Dashboard that was made in tableau.

*Please open the dashboard in full screen mode to get a better viewing experience*

In [17]:
%%HTML
<div class='tableauPlaceholder' id='viz1607519024891' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19Dashboard_15944506467040&#47;FinalDashboard&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='views&#47;Covid19Dashboard_15944506467040&#47;FinalDashboard?:language=en&amp;:embed=y&amp;:display_count=y' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19Dashboard_15944506467040&#47;FinalDashboard&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1607519024891');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>